<img src="https://user-images.githubusercontent.com/7065401/39117173-a433bf6a-46e6-11e8-8a40-b4d4d6422493.jpg"
    style="width:300px; float: right; margin: 0 40px 40px 40px;"></img>

# Data Cleansing

After dealing with many datasets I can tell you that "missing data" is not such a big deal. The best thing that can happen is to clearly see values like `np.nan`. The only thing you need to do is just use methods like `isnull` and `fillna`/`dropna` and pandas will take care of the rest.

But sometimes, you can have invalid values that are not just "missing data" (`None`, or `nan`). For example:

![separator2](https://i.imgur.com/4gX5WFr.png)

In [552]:
import numpy as np
import pandas as pd

In [553]:
df = pd.DataFrame(
    {
        "Gender": ["M", "F", "F", "D", "?", "F", "?", "M", "M"],
        "Age": [29, 30, 24, 290, 25, 19, 40, 11, 288],
    }
)
df

,Gender,Age
0,M,29
1,F,30
2,F,24
3,D,290
4,?,25
5,F,19
6,?,40
7,M,11
8,M,288


This dataframe doesn't have any "missing value", but clearly has invalid data. `290` doesn't seem like a valid age, and `D` and `?` don't correspond with any known sex category. How can you clean these not-missing, but clearly invalid values then?

# Finding Abnormal Values

The first step to clean invalid values is to **notice** them, then **identify** them and finally handle them appropriately (remove them, replace them, etc). Usually, for a "categorical" type of field (like Sex, which only takes values of a discrete set `('M', 'F')`), we start by analyzing the variety of values present. For that, we use the `unique()` method:

### use `.unique()` to identify

In [554]:
df["Gender"].unique()

array(['M', 'F', 'D', '?'], dtype=object)

In [555]:
df["Gender"].value_counts()

Gender
M    3
F    3
?    2
D    1
Name: count, dtype: int64

### replace it using `.replace()`

Clearly if you see values like `'D'` or `'?'`, it'll immediately raise your attention. Now, what to do with them? Let's say you picked up the phone, called the survey company and they told you that `'D'` was a typo and it should actually be `F`.

In [556]:
# You can use the `replace` function to replace these values:
df["Gender"].replace("D", "F")

0    M
1    F
2    F
3    F
4    ?
5    F
6    ?
7    M
8    M
Name: Gender, dtype: object

In [557]:
# It can accept a dictionary of values to replace.
# For example, they also told you that there might be a few `'?'s`, that should actually be `'M's`:
df["Gender"].replace({"D": "F", "?": "M"})

0    M
1    F
2    F
3    F
4    M
5    F
6    M
7    M
8    M
Name: Gender, dtype: object

In [558]:
# If you have many columns to replace, you could apply it at "DataFrame level":
df.replace({"Gender": {"D": "F", "?": "M"}, "Age": {290: 29}})

,Gender,Age
0,M,29
1,F,30
2,F,24
3,F,29
4,M,25
5,F,19
6,M,40
7,M,11
8,M,288


In [559]:
df.replace({"Gender": {"D": "F", "?": "M"}, "Age": {290: 29}}, inplace=True)
df

,Gender,Age
0,M,29
1,F,30
2,F,24
3,F,29
4,M,25
5,F,19
6,M,40
7,M,11
8,M,288


In the previous example, I explicitly replaced 290 with 29 (assuming it was just an extra 0 entered at data-entry phase). But what if you'd like to remove all the extra 0s from the ages columns? (example, `150 > 15`, `490 > 49`).

The first step would be to just set the limit of the "not possible" age. Is it 100? 120? Let's say that anything above 100 isn't credible for **our** dataset. Also, we do not want underage. We can then combine boolean selection with the operation:

In [560]:
df[(df["Age"] > 100) | (df["Age"] < 18)]

,Gender,Age
7,M,11
8,M,288


And we can now just divide by 10 who is older than 100, and drop who is underage

In [561]:
df = df[df["Age"] >= 18]
df.loc[df["Age"] > 100, "Age"] = df.loc[df["Age"] > 100, "Age"] / 10
df

/var/folders/0s/p6r6pj_15m5cg5q_gj8_xdjm0000gn/T/ipykernel_89121/2324351843.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[28.8]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df["Age"] > 100, "Age"] = df.loc[df["Age"] > 100, "Age"] / 10


,Gender,Age
0,M,29.0
1,F,30.0
2,F,24.0
3,F,29.0
4,M,25.0
5,F,19.0
6,M,40.0
8,M,28.8


![separator1](https://i.imgur.com/ZUWYTii.png)

# Duplicates

Checking duplicate values is extremely simple. It'll behave differently between Series and DataFrames. Let's start with Series. As an example, let's say we're throwing a fancy party and we're inviting Ambassadors from Europe. But can only invite one ambassador per country. This is our original list, and as you can see, both the UK and Germany have duplicated ambassadors:

## Duplicates in Series

In [562]:
ambassadors = pd.Series(
    [
        "France",
        "United Kingdom",
        "United Kingdom",
        "Italy",
        "Germany",
        "Germany",
        "Germany",
    ],
    index=[
        "Gérard Araud",
        "Kim Darroch",
        "Peter Westmacott",
        "Armando Varricchio",
        "Peter Wittig",
        "Peter Ammon",
        "Klaus Scharioth ",
    ],
)
ambassadors

Gérard Araud                  France
Kim Darroch           United Kingdom
Peter Westmacott      United Kingdom
Armando Varricchio             Italy
Peter Wittig                 Germany
Peter Ammon                  Germany
Klaus Scharioth              Germany
dtype: object

The two most important methods to deal with duplicates are `duplicated` (that will tell you which values are duplicates) and `drop_duplicates` (which will just get rid of duplicates):

In [563]:
ambassadors.duplicated()

Gérard Araud          False
Kim Darroch           False
Peter Westmacott       True
Armando Varricchio    False
Peter Wittig          False
Peter Ammon            True
Klaus Scharioth        True
dtype: bool

In this case `duplicated` didn't consider `'Kim Darroch'`, the first instance of the United Kingdom or `'Peter Wittig'` as duplicates. That's because, by default, it'll consider the first occurrence of the value as not-duplicate. You can change this behavior with the `keep` parameter:

In [564]:
ambassadors.duplicated(keep="last")

Gérard Araud          False
Kim Darroch            True
Peter Westmacott      False
Armando Varricchio    False
Peter Wittig           True
Peter Ammon            True
Klaus Scharioth       False
dtype: bool

In this case, the result is "flipped", `'Kim Darroch'` and `'Peter Wittig'` (the first ambassadors of their countries) are considered duplicates, but `'Peter Westmacott'` and `'Klaus Scharioth'` are not duplicates. You can also choose to mark all of them as duplicates with `keep=False`:

A similar method is `drop_duplicates`, which just excludes the duplicated values and also accepts the `keep` parameter:

In [565]:
ambassadors.drop_duplicates()

Gérard Araud                  France
Kim Darroch           United Kingdom
Armando Varricchio             Italy
Peter Wittig                 Germany
dtype: object

In [566]:
ambassadors.drop_duplicates(keep="last")

Gérard Araud                  France
Peter Westmacott      United Kingdom
Armando Varricchio             Italy
Klaus Scharioth              Germany
dtype: object

In [567]:
ambassadors.drop_duplicates(keep=False)

Gérard Araud          France
Armando Varricchio     Italy
dtype: object

## Duplicates in DataFrames

Conceptually speaking, duplicates in a DataFrame happen at "row" level. Two rows with exactly the same values are considered to be duplicates:

In [568]:
players = pd.DataFrame(
    {
        "Name": [
            "Kobe Bryant",
            "LeBron James",
            "Kobe Bryant",
            "Carmelo Anthony",
            "Kobe Bryant",
        ],
        "Pos": ["SG", "SF", "SG", "SF", "SF"],
    }
)
players

,Name,Pos
0,Kobe Bryant,SG
1,LeBron James,SF
2,Kobe Bryant,SG
3,Carmelo Anthony,SF
4,Kobe Bryant,SF


In [569]:
players.sort_values("Name", ascending=False)

,Name,Pos
1,LeBron James,SF
0,Kobe Bryant,SG
2,Kobe Bryant,SG
4,Kobe Bryant,SF
3,Carmelo Anthony,SF


In the previous DataFrame, we clearly see that Kobe is duplicated; but he appears with two different positions. What does `duplicated` say?

In [570]:
players.duplicated()

0    False
1    False
2     True
3    False
4    False
dtype: bool

Again, conceptually, "duplicated" means "all the column values should be duplicates". We can customize this with the `subset` parameter:

In [571]:
players.duplicated(subset=["Name"])

0    False
1    False
2     True
3    False
4     True
dtype: bool

And the same rules of `keep` still apply:

In [572]:
players.duplicated(subset=["Name"], keep="last")

0     True
1    False
2     True
3    False
4    False
dtype: bool

`drop_duplicates` takes the same parameters:

In [573]:
players

,Name,Pos
0,Kobe Bryant,SG
1,LeBron James,SF
2,Kobe Bryant,SG
3,Carmelo Anthony,SF
4,Kobe Bryant,SF


In [574]:
players.drop_duplicates()

,Name,Pos
0,Kobe Bryant,SG
1,LeBron James,SF
3,Carmelo Anthony,SF
4,Kobe Bryant,SF


In [575]:
players.drop_duplicates(subset=["Name"])

,Name,Pos
0,Kobe Bryant,SG
1,LeBron James,SF
3,Carmelo Anthony,SF


In [576]:
players.drop_duplicates(subset=["Name"], keep="last")

,Name,Pos
1,LeBron James,SF
3,Carmelo Anthony,SF
4,Kobe Bryant,SF


![separator1](https://i.imgur.com/ZUWYTii.png)

# Text Handling

Cleaning text values can be incredibly hard. Invalid text values involves, 99% of the time, mistyping, which is completely unpredictable and doesn't follow any pattern. Thankfully, it's not so common these days, where data-entry tasks have been replaced by machines. Still, let's explore the most common cases:

### Splitting Columns

The result of a survey is loaded and this is what you get:

In [577]:
df = pd.DataFrame(
    {
        "Data": [
            "1987_M._US _1",
            "1990?_M_UK_1",
            "1992_F_US_2",
            "1970?_M_   IT_1",
            "1985_F_I  T_2",
        ]
    }
)
df

,Data
0,1987_M._US _1
1,1990?_M_UK_1
2,1992_F_US_2
3,1970?_M_ IT_1
4,1985_F_I T_2


You know that the single columns represent the values "year, Gender, Country and number of children", but it's all been grouped in the same column and separated by an underscore. Pandas has a convenient method named `split` that we can use in these situations:

In [578]:
df["Data"].str.split("_")

0      [1987, M., US , 1]
1       [1990?, M, UK, 1]
2        [1992, F, US, 2]
3    [1970?, M,    IT, 1]
4      [1985, F, I  T, 2]
Name: Data, dtype: object

In [579]:
df["Data"].str.split("_", expand=True)

,0,1,2,3
0,1987,M.,US,1
1,1990?,M,UK,1
2,1992,F,US,2
3,1970?,M,IT,1
4,1985,F,I T,2


In [580]:
df = df["Data"].str.split("_", expand=True)
df.columns = ["Year", "Gender", "Country", "Numb_Children"]
df

,Year,Gender,Country,Numb_Children
0,1987,M.,US,1
1,1990?,M,UK,1
2,1992,F,US,2
3,1970?,M,IT,1
4,1985,F,I T,2


but seems like the data still looks ugly. </br>
How do we know?

In [581]:
print(df["Year"].unique())
print(df["Gender"].unique())
print(df["Country"].unique())
print(df["Numb_Children"].unique())

['1987' '1990?' '1992' '1970?' '1985']
['M.' 'M' 'F']
['US ' 'UK' 'US' '   IT' 'I  T']
['1' '2']


### strip the spaces
You can also check which columns contain a given value with the `contains` method </br>
Removing blank spaces (like in `'US '` or `'I  T'` can be achieved with `strip` (`lstrip` and `rstrip` also exist) or just `replace`:

In [582]:
df["Country"].str.contains(" ")

0     True
1    False
2    False
3     True
4     True
Name: Country, dtype: bool

In [583]:
df["Country"].str.strip()

0      US
1      UK
2      US
3      IT
4    I  T
Name: Country, dtype: object

In [584]:
df["Country"].str.replace(" ", "")

0    US
1    UK
2    US
3    IT
4    IT
Name: Country, dtype: object

In [585]:
df["Country"] = df["Country"].str.replace(" ", "")
print(df["Country"].unique())
df

['US' 'UK' 'IT']


,Year,Gender,Country,Numb_Children
0,1987,M.,US,1
1,1990?,M,UK,1
2,1992,F,US,2
3,1970?,M,IT,1
4,1985,F,IT,2


### `.contains` beware regular expression

You can also check which columns contain a given value with the `contains` method:
- And here you need to use `\\?` not `?`
- Same way, you need to use `\\.` not `.`

[`contains`](http://pandas.pydata.org/pandas-docs/version/0.22.0/generated/pandas.Series.str.contains.html) takes a regex/pattern as first value, so we need to escape the `?` symbol or `.` as it has a special meaning for these patterns.

In [586]:
# df["Year"].str.contains("?")  # error!! you need to use \\?
df["Year"].str.contains("\\?")

0    False
1     True
2    False
3     True
4    False
Name: Year, dtype: bool

In [587]:
# print(df["Gender"].str.contains(".")) # no error. but you will get all True
print(df["Gender"].str.contains("\\."))

0     True
1    False
2    False
3    False
4    False
Name: Gender, dtype: bool


In [588]:
df["Year"] = df["Year"].str.replace("?", "")
df["Gender"] = df["Gender"].str.replace(".", "")
df

,Year,Gender,Country,Numb_Children
0,1987,M,US,1
1,1990,M,UK,1
2,1992,F,US,2
3,1970,M,IT,1
4,1985,F,IT,2


In [589]:
print(df["Numb_Children"].mean(), df["Numb_Children"].std().round(2))  # error

TypeError: Could not convert string '11212' to numeric

In [590]:
df["Numb_Children"] = df["Numb_Children"].astype(int)
print(df["Numb_Children"].mean(), df["Numb_Children"].std().round(2))

1.4 0.55
